<a href="https://colab.research.google.com/github/sudip234-source/PyTorch-Tutorial/blob/main/mySimpleNN_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [765]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [766]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [767]:
df.shape

(569, 33)

In [768]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [769]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [770]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [771]:
X_train

array([[-0.13785314, -0.70322906, -0.18519331, ..., -0.45063162,
        -0.16487748, -0.36436544],
       [ 0.06664953, -0.65433294,  0.07060995, ...,  0.36432946,
         0.22353814,  0.15998741],
       [ 0.92887696, -0.00704145,  0.90927638, ...,  1.15140555,
         0.02697153, -0.31897848],
       ...,
       [-0.93375539, -2.24462102, -0.94539739, ..., -1.06545048,
        -1.29081102, -0.40868447],
       [-1.23221873, -0.26316398, -1.26725314, ..., -1.75552882,
        -1.57543947, -1.01740366],
       [ 0.09981212,  1.90689237,  0.17389202, ...,  0.81108884,
         2.05553894,  2.64078511]])

In [772]:
y_test[:2]

,diagnosis
183,B
55,B


In [773]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [774]:
y_test[:2]

array([0, 0])

In [775]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [776]:
X_train_tensor.shape,y_test_tensor.shape

(torch.Size([455, 30]), torch.Size([114]))

## Defining The Model

In [777]:
class mySimpleNN:
  def __init__(self, X):
    self.w1 = torch.rand(X.shape[1], 16, dtype=torch.float32, requires_grad=True)  # input to hidden1
    self.b1 = torch.rand(16, dtype=torch.float32, requires_grad=True)

    self.w2 = torch.rand(16, 8, dtype=torch.float32, requires_grad=True)           # hidden1 to hidden2
    self.b2 = torch.rand(8, dtype=torch.float32, requires_grad=True)

    self.w3 = torch.rand(8, 1, dtype=torch.float32, requires_grad=True)            # hidden2 to output
    self.b3 = torch.rand(1, dtype=torch.float32, requires_grad=True)

  def forward(self, X):
    z1 = torch.matmul(X, self.w1) + self.b1      # (batch, 16)
    o1 = torch.sigmoid(z1)

    z2 = torch.matmul(o1, self.w2) + self.b2     # (batch, 8)
    o2 = torch.sigmoid(z2)

    z3 = torch.matmul(o2, self.w3) + self.b3     # (batch, 1)
    o3 = torch.sigmoid(z3)

    return o3

  def parameters(self):
    return [self.w1, self.b1, self.w2, self.b2, self.w3, self.b3]

  def calc_loss(self, y_pred, y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
    return loss


## Training pipeline

In [778]:
from re import X
X_train_tensor = X_train_tensor.to(torch.float32)
y_train_tensor = y_train_tensor.to(torch.float32)
X_test_tensor = X_test_tensor.to(torch.float32)
y_test_tensor = y_test_tensor.to(torch.float32)

In [779]:
epochs = 500
learning_rate = 0.06

In [780]:
model = mySimpleNN(X_train_tensor)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = model.calc_loss(y_pred, y_train_tensor)
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            param.grad.zero_()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")



Epoch 1, Loss: 2.304030656814575
Epoch 2, Loss: 2.1686134338378906
Epoch 3, Loss: 2.036098003387451
Epoch 4, Loss: 1.9070535898208618
Epoch 5, Loss: 1.7821295261383057
Epoch 6, Loss: 1.6620486974716187
Epoch 7, Loss: 1.5475850105285645
Epoch 8, Loss: 1.4395347833633423
Epoch 9, Loss: 1.3386667966842651
Epoch 10, Loss: 1.2456661462783813
Epoch 11, Loss: 1.1610689163208008
Epoch 12, Loss: 1.0852067470550537
Epoch 13, Loss: 1.0181635618209839
Epoch 14, Loss: 0.9597631692886353
Epoch 15, Loss: 0.9095879793167114
Epoch 16, Loss: 0.8670234084129333
Epoch 17, Loss: 0.8313199877738953
Epoch 18, Loss: 0.8016601800918579
Epoch 19, Loss: 0.7772186398506165
Epoch 20, Loss: 0.7572064995765686
Epoch 21, Loss: 0.7409029603004456
Epoch 22, Loss: 0.7276705503463745
Epoch 23, Loss: 0.7169597744941711
Epoch 24, Loss: 0.708305835723877
Epoch 25, Loss: 0.7013217210769653
Epoch 26, Loss: 0.6956890225410461
Epoch 27, Loss: 0.691146731376648
Epoch 28, Loss: 0.6874838471412659
Epoch 29, Loss: 0.684528946876525

In [781]:
model.b1

tensor([0.5306, 0.8787, 0.6963, 0.7762, 0.3511, 0.3487, 0.4846, 0.8986, 0.7563,
        0.6134, 0.4246, 0.4816, 0.9517, 0.5517, 0.8145, 0.8048],
       requires_grad=True)

In [782]:
model.w1.to_sparse().values()[:10]

tensor([0.0614, 0.8996, 0.3596, 0.2180, 0.0529, 0.9597, 0.9156, 0.7056, 0.8865,
        0.1200], grad_fn=<SliceBackward0>)

In [783]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.7105262875556946
